# Data exploration

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib3

Data extracted on Thursday 28th, October 2021 at 1:03 PM from Crunchbase Research.

In [3]:
extracted_year = 2021

## Load tables

In [100]:
# Tables we will use
people_df = pd.read_csv('data/people.csv')
organizations_df = pd.read_csv('data/organizations.csv')
jobs_df = pd.read_csv('data/jobs.csv')
investors_df = pd.read_csv('data/investors.csv')
funding_rounds_df = pd.read_csv('data/funding_rounds.csv')
degrees_df = pd.read_csv('data/degrees.csv')

category_groups_df = pd.read_csv('data/category_groups.csv')

In [ ]:
# Useless tables
organization_descriptions_df = pd.read_csv('data/organization_descriptions.csv')
people_descriptions_df = pd.read_csv('data/people_descriptions.csv')
check_sum_df = pd.read_csv('data/checksum.csv')
event_appearances_df = pd.read_csv('data/event_appearances.csv')
ipos_df = pd.read_csv('data/ipos.csv')
acquisitions_df = pd.read_csv('data/acquisitions.csv')
investments_df = pd.read_csv('data/investments.csv')
funds_df = pd.read_csv('data/funds.csv')
org_parents = pd.read_csv('data/org_parents.csv')
events = pd.read_csv('data/events.csv')
investment_partners_df = pd.read_csv('data/investment_partners.csv')

## Preliminary table selection

The goal of this step is to figure which tables contain meaningful data for our analysis. The tables that do not make the cut will not be used further.

### People-related tables

In [ ]:
# print(people_df.columns.values)
# people_df.head()

The table <code>people_df</code> contains potentially good information like rank, region, city, etc. Furthermore, we can join this table with some other ones like <code>organizations_df</code> or <code>investors_df</code>. We will keep using this so far.

In [ ]:
#print(people_descriptions_df.columns.values)
#people_descriptions_df.head()

The table <code>people_descriptions_df</code> mostly contains text description of what each company does. While this might be useful for a model that leverages NLP, that is out of the scope of our project

In [ ]:
# print(ipos_df.columns.values)
#ipos_df.head()

The table <code>jobs_df</code> contains useful information regarding the people that work on the company. We can leverage this table to find information regarding the founders.

In [ ]:
# print(degrees_df.columns.values)
# degrees_df.head()

The table <code>degrees_df</code> contains information regarding the degrees of people. We can use this information to see where founders and early employees went to school.

### Organization-related tables

In [ ]:
# print(organizations_df.columns.values)
# organizations_df.head()

The table <code>organizations_df</code> contains potentially good information like rank, status, category_list, total_funding etc. Furthermore, we can join this table with some other ones like <code>people_df</code> or <code>investors_df</code>. We will keep using this so far.

In [ ]:
# print(organization_descriptions_df.columns.values)
# organization_descriptions_df.head()

The table <code>organization_descriptions_df</code> mostly contains text description of what each company does. While this might be useful for a model that leverages NLP, that is out of the scope of our project.

In [ ]:
# print(org_parents_df.columns.values)
# org_parents_df.head()

The table <code>org_parents_df</code> mostly contains information about parent and child companies. This information is not relevant for our analysis.

In [ ]:
# print(jobs_df.columns.values)
# jobs_df.head()

The table <code>ipos_df</code> is an extremely important one as it contains useful information about companies that exited through an IPO.

In [ ]:
# print(acquisitions_df.columns.values)
# acquisitions_df.head()

We do not need the table <code>acquisitions_df</code> as <code>organizations_df</code> already tells us whether a company has been acquired.

### Investment/funding-related tables

In [ ]:
# print(investors_df.columns.values)
# investors_df.head()

The table <code>investors_df</code> contains information regarding investors including domain, total money invested, investment count, etc. We will use it in our analysis.

In [ ]:
# print(investments_df.columns.values)
# investments_df.head()

The table <code>investments_df</code> contains information regarding particular investments including funding round, investor name, etc. While this information might be useful, it is already included in other tables like <code>funding_rounds_df</code> and <code>investors_df</code>

In [ ]:
# print(funding_rounds_df.columns.values)
# funding_rounds_df.head()

The table <code>funding_rounds_df</code> contains information regarding particular founding rounds including investor name, investor count, year, raised amount, etc. This table might come in handy later.

In [ ]:
# print(funds_df.columns.values)
# funds_df.head()

The table <code>funds_df</code> contains information regarding investment funds including domain, total money raised, investment count, etc. This information is not relevant for our analysis as it does not shed any light on investment decisions.

In [ ]:
# print(investment_partners_df.columns.values)
# investment_partners_df.head()

The table <code>investment_partners</code> contains information regarding the partners that led each investment. While this information might be really interesting, we are not looking for this level of granularity in our analysis. Because of that, we will not use this table moving forward.

### Events-related tables

The following tables include information about different entrepreneurship/tech events awnd which companies attended. We beleive

In [ ]:
# print(event_appearances_df.columns.values)
# event_appearances_df.head()

In [ ]:
# print(events_df.columns.values)
# events.head()

## Preliminary data cleaning

### Some helper functions

In [ ]:
# Define a function that checks if an entry is positions
def check_position(entry, positions,s=' '):
    ans = [word.lower() in positions for word in str(entry).split(s)]
    return min(1, sum(ans))


# Define a function that checks if a company was founded within a range
def founded_on(entry, years):
    year = int(entry[0:4])
    return years[0] <= year and year <= years[1]
    
# Define a function that checks if a company url is still active
http = urllib3.PoolManager()
def still_active(url):
    
    try:
        r = http.request('GET', url)
        
        print(r.status)
        return r.status == 200
    except:
        return False

### People_df

In [103]:
people_df = people_df.drop(columns=['permalink','cb_url','created_at','updated_at',
                                    'state_code','region','city','facebook_url',
                                    'linkedin_url','twitter_url','logo_url'])

### Organizations_df

In [104]:
organizations_df = organizations_df.drop(columns=['permalink','cb_url','created_at','updated_at',
                                                  'legal_name','phone','short_description','email',
                                                  'phone','facebook_url','linkedin_url','twitter_url',
                                                  'logo_url','alias1','alias2','alias3','primary_role',
                                                  'num_exits','rank','total_funding','total_funding_currency_code']) 

In [ ]:
# Get organizations in the USA
cond1 = organizations_df['country_code'] == 'USA'

# Organizations_dfies founded between 1995 and 2000 in the USA

# We drop all those with nan values
organizations_df = organizations_df.dropna(subset=['founded_on'])
founded_95_15 = lambda x: founded_on(x,[1995,2015])
cond2 = organizations_df['founded_on'].map(founded_95_15)

organizations_df = organizations_df[cond1 & cond2]

# Only keep companies, filter out all investment funds
#check_company = lambda x: check_position(x,['company'],',')

cond = organizations_df['roles'].map(check_company).astype(bool)
organizations_df = organizations_df[cond]

# Get which website domains are still working
organizations_df['active_homepage'] = organizations_df['homepage_url'].map(still_active).astype(bool)

### Jobs_df

In [ ]:
#jobs_df = jobs_df.drop(columns=['permalink','cb_url','rank','created_at','updated_at'])
jobs_df['started_on']= pd.to_datetime(jobs_df['started_on'],errors = 'coerce')

In [ ]:
# We drop titles with NaN
jobs_df = jobs_df[jobs_df['title'].notna()]

# Define functions to check if_founder or if_ceo
check_founder = lambda x: check_position(x, ['founder','co-founder'])
check_ceo = lambda x: check_position(x, ['ceo'])

# Create new columns for is_founder and is_ceo
jobs_df['is_founder'] = jobs_df['title'].map(check_founder).astype(bool)
jobs_df['is_current_ceo'] = jobs_df['title'].map(check_ceo) & jobs_df['is_current']

# Drop all but founders and current CEO
cond = jobs_df['is_founder']  | jobs_df['is_current_ceo']
jobs_df = jobs_df[cond.astype(bool)]

# We want to calculate how many are previously founders
founders_df = jobs_df[jobs_df['is_founder'] == 1]
serial_founders_df = founders_df.groupby(by=['person_uuid'])\
                                .agg({"org_uuid": pd.Series.nunique, 'started_on':'min'})\
                                .reset_index()\
                                .rename(columns={'org_uuid':'number_founded', 'started_on':'first_venture_on'})

# Merge tables
jobs2_df = jobs_df.merge(serial_founders_df,how='left',on='person_uuid')

# We determine whether a founder was a veteran founder at the time each venture was founded
cond1 = jobs2_df['is_founder'] == True
cond2 = jobs2_df['number_founded'] > 1
cond3 = jobs2_df['first_on'] < jobs2_df['started_on']

jobs2_df['veteran_founder'] = cond1 & cond2 & cond3

# Get rid of unnecessary columns
jobs2_df = jobs2_df.drop(columns=['uuid','name','type','person_name','org_name','ended_on','title','job_type'])

In [ ]:
jobs2_df

### Investors_df

In [106]:
investors_df = investors_df.drop(columns=['permalink','cb_url','rank','created_at',
                                          'updated_at','domain','country_code','state_code','region','city',
                                          'founded_on','closed_on','facebook_url',
                                          'linkedin_url','twitter_url','logo_url',
                                          'total_funding_usd', 'total_funding', 'total_funding_currency_code'])

In [107]:
#investors_df

### Funding_rounds_df

In [112]:
funding_rounds_df = funding_rounds_df.drop(columns=['permalink','cb_url','rank','created_at',
                                                    'updated_at','country_code','state_code',
                                                    'region','city','announced_on','post_money_valuation',
                                                    'post_money_valuation_currency_code',
                                                   'raised_amount', 'raised_amount_currency_code','post_money_valuation_usd'])

In [113]:
# Exclude all rounds taking place after Series B
rounds = ['angel', 'seed', 'series_a', 'series_b']
check_rounds = lambda x: x in rounds

cond = funding_rounds_df['investment_type'].map(check_rounds)
funding_rounds_df = funding_rounds_df[cond]

# We create an indicator variable to see if an investment round had a lead_investor
# True means the round had a lead investor
funding_rounds_df['lead_investor_ind'] = ~(funding_rounds_df['lead_investor_uuids'].isna())

In [114]:
# Merge tables to include lead_investor_investment count
funding_rounds_df

,uuid,name,type,investment_type,raised_amount_usd,investor_count,org_uuid,org_name,lead_investor_uuids,lead_investor_ind
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Facebook,funding_round,angel,500000.0,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,3f47be49-2e32-8118-01a0-31685a4d0fd7,True
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,funding_round,series_a,12700000.0,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36,True
2,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,funding_round,series_b,27500000.0,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-...",True
3,bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,funding_round,series_b,10500000.0,2.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,Photobucket,NaN,False
4,60e6afd9-1215-465a-dd17-0ed600d4e29b,Series A - Geni,funding_round,series_a,NaN,1.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,fb2f8884-ec07-895a-48d7-d9a9d4d7175c,True
...,...,...,...,...,...,...,...,...,...,...
438005,bbf0cd68-ed21-4c39-b941-b746aa24ee90,Seed Round - Nectir,funding_round,seed,2250000.0,1.0,7a545a66-56b1-4d98-ab0c-306a8b68de37,Nectir,aa4c2058-ddae-4f1c-b0f0-6ba6b07a1a19,True
438022,b4bac7fe-dac0-4e5b-8766-f0d9cae0a4f8,Seed Round - qBotica,funding_round,seed,1000000.0,NaN,76b76eaa-331f-9ce3-3722-fdc82e296cf2,qBotica,NaN,False
438024,a404c3ae-1e2c-45b8-8bea-ae8db54b321f,Seed Round - PipeKit,funding_round,seed,NaN,1.0,308b85fe-5e63-43ef-9750-21c7f10246a9,PipeKit,NaN,False
438025,012ce68a-3ba7-4ac3-bd8d-1c5501022382,Series A - Buffalo Market,funding_round,series_a,19749100.0,6.0,9357d833-5bbc-4224-9a42-3d81c36f6476,Buffalo Market,"bd699a78-d3ac-4858-9b37-ff62a2112147,893f5705-...",True


### Degrees_df

In [115]:
degrees_df = degrees_df.drop(columns=['permalink','cb_url','rank','created_at',
                                      'updated_at','institution_uuid','institution_name','subject'])

In [116]:
# Convert our dates into date_data
# Convert invalid dates to NaT
degrees_df['completed_on'] = pd.to_datetime(degrees_df['completed_on'],errors = 'coerce')

# Find whether a person has completed a degree, how many degrees they have attempted, and the earliest completion date
degrees_df = degrees_df.groupby(by=['person_uuid']).agg({'uuid':'count', 'is_completed':'sum', 'completed_on':'min'})
degrees_df = degrees_df.rename(columns={'uuid':'num_degs_attempted', 'is_completed':'num_degs_finished', 'completed_on':'first_deg_completed_date'})

In [117]:
#degrees_df

### Category_groups_df

In [118]:
category_groups_df = category_groups_df.drop(columns=['permalink','cb_url','rank','created_at','updated_at'])

In [119]:
category_groups_df

,uuid,name,type,category_groups_list
0,f9b14a15-5517-8f38-0562-729ebb54dfdb,Homeless Shelter,category,Other
1,f8320fcf-b657-37d2-1495-daa3ad888ece,Freemium,category,Other
2,f0193199-a968-b457-eb49-95344e22a5ce,Industrial,category,Manufacturing
3,ed8217ff-bd11-26fe-d82e-eb98a276dc2e,Innovation Management,category,Professional Services
4,e9794581-9547-2150-8185-7b747f5c9913,Lighting,category,Hardware
...,...,...,...,...
739,68413bd1-e42d-4cf1-815b-62e64a6b922a,Quantum Computing,category,Science and Engineering
740,3e0e2772-75e9-4f7b-aa6d-a8dc3dc9e7c6,Marine Technology,category,Science and Engineering
741,ce11fd97-65a0-4f8c-b385-2ca48f444479,Smart Cities,category,Real Estate
742,b561fa84-cb20-4a82-a6bc-fea613d74cdd,Last Mile Transportation,category,Transportation


### Merging to obtain final dataset

In [125]:
organizations_df

,uuid,name,type,roles,domain,homepage_url,country_code,state_code,region,city,...,postal_code,status,category_list,category_groups_list,num_funding_rounds,total_funding_usd,founded_on,last_funding_on,closed_on,employee_count
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,company,wetpaint.com,http://www.wetpaint.com/,USA,NY,New York,New York,...,10010,acquired,"Publishing,Social Media,Social Media Management","Content and Publishing,Internet Services,Media...",3.0,3.975000e+07,2005-06-01,2008-05-19,NaN,51-100
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,"investor,company",zoho.com,https://www.zoho.com/,USA,CA,California,Pleasanton,...,94588,operating,"Cloud Computing,Collaboration,CRM,Developer To...","Administrative Services,Information Technology...",NaN,NaN,1996-09-15,NaN,NaN,1001-5000
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,company,digg.com,http://www.digg.com,USA,NY,New York,New York,...,NaN,acquired,"Internet,Social Media,Social Network","Internet Services,Media and Entertainment",6.0,4.900000e+07,2004-10-11,2016-09-13,NaN,51-100
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,investor,omidyar.com,http://www.omidyar.com,USA,CA,California,Redwood City,...,94063,operating,"Enterprise Software,Financial Services,Venture...","Financial Services,Lending and Investments,Sof...",NaN,NaN,2004-01-01,NaN,NaN,101-250
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,"investor,company",facebook.com,http://www.facebook.com,USA,CA,California,Menlo Park,...,94025,ipo,"Mobile Apps,Photo Sharing,Social Media,Social ...","Apps,Content and Publishing,Internet Services,...",16.0,1.612282e+10,2004-02-04,2014-10-20,NaN,10000+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642158,9ed8a203-c5a0-496a-adda-25a0cbb90fb3,McGuire Memorial,organization,company,mcguirememorial.org,http://mcguirememorial.org,USA,PA,Pennsylvania,New Brighton,...,15066,operating,"Assisted Living,Education,Health Care,Non Prof...","Education,Health Care,Other,Real Estate",NaN,NaN,1963-01-01,NaN,NaN,251-500
1642159,a8b940d2-89f3-46f9-80ed-69cf4ceea3ac,Success Chain,organization,company,successchain.net,https://www.successchain.net,GBR,NaN,Bournemouth,Bournemouth,...,NaN,operating,"Business Development,Consulting,SaaS","Professional Services,Software",NaN,NaN,2020-01-01,NaN,NaN,1-10
1642160,ffc73558-8701-4530-ae8b-293eabf9a7b8,Capital Estate,organization,company,capitalestate.mx,http://www.capitalestate.mx,MEX,NaN,Jalisco,Guadalajara,...,NaN,operating,Real Estate,Real Estate,NaN,NaN,2020-05-01,NaN,NaN,1-10
1642161,440cc557-5891-4b6b-935a-d90569cff9d8,FABAY,organization,company,fabay.com.tr,https://fabay.com.tr,TUR,NaN,Mugla,Mugla,...,NaN,operating,"Civil Engineering,Construction,Project Managem...","Administrative Services,Other,Real Estate,Scie...",NaN,NaN,NaN,NaN,NaN,1001-5000


In [ ]:
"""
# Create a bar plot for companies per state

to_plot_states = organizations_df.groupby(by='state_code').count().reset_index()[['state_code','uuid']]
sns.set_theme(style="whitegrid")
fig, ax = plt.subplots(1,1, figsize=(10,5))

ax = sns.barplot(x="state_code", y="uuid", data=to_plot_states)
ax.set_xticklabels(to_plot_states["state_code"],rotation=45,fontsize=10)

plt.show()
"""